In [1]:
%pip install -qU langgraph langchain langchain_openai langchain_experimental pymupdf tiktoken python-dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.1 which is incompatible.
langchain-qdrant 0.1.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
# from helper.file_loader import FileLoader

load_dotenv()

open_ai_key = os.getenv("OPENAI_API_KEY")
langchain_key = os.getenv("LANGCHAIN_API_KEY")

llm = ChatOpenAI(api_key=open_ai_key, model="gpt-4o")

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

loader1 = PyMuPDFLoader("https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf")
document1 = loader1.load()

loader2 = PyMuPDFLoader("https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.600-1.pdf")
document2 = loader2.load()

document = document1 + document2